In [ ]:
#!conda install -y llama_index
#!conda  install llama_index

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import torch
from string import Template
from pathlib import Path
import json
from tqdm.auto import tqdm
import warnings
warnings.simplefilter("ignore")

import torch

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
with open("/kaggle/input/telecom-dataset/TeleQnA_training.txt", "r") as f:
    train_data = f.read()
    
with open("/kaggle/input/telecom-dataset/TeleQnA_testing1.txt", "r") as f:
    test_data = f.read()

In [ ]:
test_data=json.loads(test_data)
train_data=json.loads(train_data)

In [ ]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
         })
    return pd.DataFrame(data)

test_df=dict_to_frame(test_data)
train_df=dict_to_frame(train_data)
test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)
train_df

In [ ]:
preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('$preamble\n\n$prompt\n\nA) $a\nB) $b\nC) $c\nD) $d')

In [ ]:
def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d)
    
    return input_text

In [ ]:
print(format_input(test_df, 3))

In [ ]:
ss=pd.read_csv('/kaggle/input/dataset/SampleSubmission (22).csv')
ans=pd.read_csv('/kaggle/input/dataset/Q_A_ID_training.csv')

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)


In [ ]:
if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def get_ans():
    ans_lst=[]
    for i in tqdm(range(test_df.shape[0])):
        inputs = tokenizer(format_input(test_df, i), return_tensors="pt", return_attention_mask=False).to(device)
        outputs = model.generate(**inputs,pad_token_id=tokenizer.pad_token_id, max_length=200)
        text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        ans=extract_answer(text)[0]
        ans_lst.append(ans)
    return ans_lst
ans_lst=get_ans()


In [ ]:
map_ans={'A':1, 'B':2, 'C':3, 'D':4}

In [ ]:
#sub=pd.read_csv('/kaggle/input/telecom-networks-5g-data/sub (10).csv')

In [ ]:
sub=pd.DataFrame({'Question_ID':test_df['question_id'].values, 'Answer_ID':ans_lst})
sub['Answer_ID']=sub['Answer_ID'].map(map_ans)
sub['Answer_ID']=sub['Answer_ID'].fillna(3).astype(int)
subzindi=pd.concat([sub, ss[~ss['Question_ID'].isin(sub['Question_ID'])]])
subzindi.to_csv('subllm.csv', index=False)